<a href="https://colab.research.google.com/github/IshtiSikder/513-replicating-results/blob/edit/513_final_project_replicating_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
#EDITS
#Install gurobi
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [183]:
#Import necessary libraries
import sys
import numpy as np
import pandas as pd
from numpy import linalg as la
import gurobipy as gp
from gurobipy import *

In [184]:
#Activate gurobi license (only if run on google colab)
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', 'dcee8c55-6742-4816-bae1-a0742c8f71a9')
e.setParam('WLSSECRET', '70f89e9f-2426-41b9-abe7-88c4b6090a45')
e.setParam('LICENSEID',820825)
e.start()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 820825
Academic license - for non-commercial use only - registered to isikder@uncc.edu


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=820825>

In [185]:
#Setup the optimization model
New_Model = gp.Model("New Model",env=e) 

In [186]:
data = pd.DataFrame({'pre-demand':[100,150,0,0,0,0],'pre-demand prob':[0.5,0.5,0,0,0,0],'demand':[650,750,850,950,1050,1150],'demand prob':[0.17,0.27,0.23,0.17,0.12,0.04]})
data

,pre-demand,pre-demand prob,demand,demand prob
0,100,0.5,650,0.17
1,150,0.5,750,0.27
2,0,0.0,850,0.23
3,0,0.0,950,0.17
4,0,0.0,1050,0.12
5,0,0.0,1150,0.04


In [187]:
K = range(1,101)
T = [1,2]

In [188]:
Q = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
v = New_Model.addVars(K,[0,1,2],lb=0,vtype=GRB.INTEGER)
u = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
x = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
q = New_Model.addVars(K,lb=0,vtype=GRB.INTEGER)
c = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)
h = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)
s = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)

In [189]:
'''

for k in K:
  for t in T:
      if 1<=k<=6:
        x[k,t]=data['demand'][0] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][0]
      elif 7<=k<=12:
        x[k,t]=data['demand'][1] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][1]
      elif 13<=k<=18:
        x[k,t]=data['demand'][2] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][2]
      elif 19<=k<=24:
        x[k,t]=data['demand'][3] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][3]
      elif 25<=k<=30:
        x[k,t]=data['demand'][4] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][4]
      else:
        x[k,t]=data['demand'][5] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][5]
'''  

"\n\nfor k in K:\n  for t in T:\n      if 1<=k<=6:\n        x[k,t]=data['demand'][0] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n        q[k,t]=data['demand prob'][0]\n      elif 7<=k<=12:\n        x[k,t]=data['demand'][1] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n        q[k,t]=data['demand prob'][1]\n      elif 13<=k<=18:\n        x[k,t]=data['demand'][2] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n        q[k,t]=data['demand prob'][2]\n      elif 19<=k<=24:\n        x[k,t]=data['demand'][3] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n        q[k,t]=data['demand prob'][3]\n      elif 25<=k<=30:\n        x[k,t]=data['demand'][4] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n        q[k,t]=data['demand prob'][4]\n      else:\n        x[k,t]=data['demand']

In [190]:
Q_prime = New_Model.addVars([1,2],[0,1,2,3,4,5,6],lb=0,vtype=GRB.INTEGER)
u_prime = New_Model.addVars([2],[1,2,3,4,5,6],lb=0,vtype=GRB.INTEGER)
v_prime = New_Model.addVars([2],[1,2,3,4,5,6],lb=0,vtype=GRB.INTEGER)

In [191]:
#bin = New_Model.addVars(K,[0,1,2],vtype=GRB.BINARY)

In [192]:
c[1] = 30
c[2] = 4*c[1]
s[1] = 300
s[2] = s[1]

for t in T:
  h[t] = c[t]/2

In [193]:
for k in K:
  hur_dem = np.random.choice(list(data['demand']), 1, list(data['demand prob']))[0]
  q[k] = data.loc[data.index[data['demand']==hur_dem].tolist()[0],'demand prob']
  for t in T:
    #if t == 1:
    x[k,t] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0] + hur_dem/2
'''
    else:
      x[k,t] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0] 
'''

"\n    else:\n      x[k,t] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0] \n"

In [194]:
New_Model.addConstrs(v[k,0]==0 for k in K)
#New_Model.addConstrs(bin[k,0]==0 for k in K)
#New_Model.addConstrs(bin[k,2]== bin[k,1]for k in K)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

In [195]:
New_Model.addConstrs(Q[k,1] == Q_prime[1,0] for k in K)

'''
New_Model.addConstrs(Q[k,2] == Q_prime[2,1] for k in K[:6])
New_Model.addConstrs(Q[k,2] == Q_prime[2,2] for k in K[6:12])
New_Model.addConstrs(Q[k,2] == Q_prime[2,3] for k in K[12:18])
New_Model.addConstrs(Q[k,2] == Q_prime[2,4] for k in K[18:24])
New_Model.addConstrs(Q[k,2] == Q_prime[2,5] for k in K[24:30])
New_Model.addConstrs(Q[k,2] == Q_prime[2,6] for k in K[30:36])
'''

'\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,1] for k in K[:6])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,2] for k in K[6:12])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,3] for k in K[12:18])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,4] for k in K[18:24])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,5] for k in K[24:30])\nNew_Model.addConstrs(Q[k,2] == Q_prime[2,6] for k in K[30:36])\n'

In [196]:
'''
New_Model.addConstrs(u[k,2] == u_prime[2,1] for k in K[:6])
New_Model.addConstrs(u[k,2] == u_prime[2,2] for k in K[6:12])
New_Model.addConstrs(u[k,2] == u_prime[2,3] for k in K[12:18])
New_Model.addConstrs(u[k,2] == u_prime[2,4] for k in K[18:24])
New_Model.addConstrs(u[k,2] == u_prime[2,5] for k in K[24:30])
New_Model.addConstrs(u[k,2] == u_prime[2,6] for k in K[30:36])
'''

'\nNew_Model.addConstrs(u[k,2] == u_prime[2,1] for k in K[:6])\nNew_Model.addConstrs(u[k,2] == u_prime[2,2] for k in K[6:12])\nNew_Model.addConstrs(u[k,2] == u_prime[2,3] for k in K[12:18])\nNew_Model.addConstrs(u[k,2] == u_prime[2,4] for k in K[18:24])\nNew_Model.addConstrs(u[k,2] == u_prime[2,5] for k in K[24:30])\nNew_Model.addConstrs(u[k,2] == u_prime[2,6] for k in K[30:36])\n'

In [197]:
'''
New_Model.addConstrs(v[k,2] == v_prime[2,1] for k in K[:6])
New_Model.addConstrs(v[k,2] == v_prime[2,2] for k in K[6:12])
New_Model.addConstrs(v[k,2] == v_prime[2,3] for k in K[12:18])
New_Model.addConstrs(v[k,2] == v_prime[2,4] for k in K[18:24])
New_Model.addConstrs(v[k,2] == v_prime[2,5] for k in K[24:30])
New_Model.addConstrs(v[k,2] == v_prime[2,6] for k in K[30:36])
'''

'\nNew_Model.addConstrs(v[k,2] == v_prime[2,1] for k in K[:6])\nNew_Model.addConstrs(v[k,2] == v_prime[2,2] for k in K[6:12])\nNew_Model.addConstrs(v[k,2] == v_prime[2,3] for k in K[12:18])\nNew_Model.addConstrs(v[k,2] == v_prime[2,4] for k in K[18:24])\nNew_Model.addConstrs(v[k,2] == v_prime[2,5] for k in K[24:30])\nNew_Model.addConstrs(v[k,2] == v_prime[2,6] for k in K[30:36])\n'

In [198]:
#New_Model.addConstrs(Q[k,t]+bin[k,t-1]*v[k,t-1]+(1-bin[k,t])*u[k,t]-bin[k,t]*v[k,t] == x[k,t] for k in K for t in T)

In [199]:
New_Model.addConstrs(Q[k,t]+v[k,t-1]+u[k,t]-v[k,t] == x[k,t] for k in K for t in T)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (6, 1): <gurobi.Constr *Awaiting Model Update*>,
 (6, 2): <gurobi.Constr *Awaiting Model Update*>,
 (7, 1): <gurobi.Constr *Awaiting Model Update*>,
 (7, 2): <gurobi.Constr *Awaiting Model Update*>,
 (8, 1): <gurobi.Constr *Awaiting Model Update*>,
 (8, 2): <gurobi.Constr *Awaiting Model Update*>,
 (9, 1): <gurobi.Constr *Awaiting Model Update*>,
 (9, 2): <gurobi.Constr *Awaiting Model Update*>,
 (10, 1): <gurobi.Constr *Awaiting Model Update*>,
 (10, 2): <gurobi.Constr *Awaiting Model Update*>

In [200]:
obj = 0

for t in T:
  for k in K:
    obj += q[k]*(c[t]*Q[k,t] + h[t]*v[k,t] + s[t]*u[k,t])

New_Model.setObjective(obj, GRB.MINIMIZE)
New_Model.optimize()


print('Obj: %g' % New_Model.ObjVal)
print('Expected value:', New_Model.ObjVal/len(K))

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to isikder@uncc.edu
Optimize a model with 400 rows, 1032 columns and 1100 nonzeros
Model fingerprint: 0xba215e4d
Variable types: 0 continuous, 1032 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 7e+02]
Found heuristic solution: objective 3955665.0000
Presolve removed 354 rows and 985 columns
Presolve time: 0.00s
Presolved: 46 rows, 47 columns, 92 nonzeros
Variable types: 0 continuous, 47 integer (0 binary)
Found heuristic solution: objective 1992406.2000

Root relaxation: objective 8.636887e+05, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bound

In [201]:
for i in [1,2]:
  for j in [0,1,2,3,4,5,6]:
    print(i,j,Q_prime[i,j].X)

1 0 1050.0
1 1 -0.0
1 2 -0.0
1 3 -0.0
1 4 -0.0
1 5 -0.0
1 6 -0.0
2 0 -0.0
2 1 -0.0
2 2 -0.0
2 3 -0.0
2 4 -0.0
2 5 -0.0
2 6 -0.0


In [202]:
for k in K:
  for t in T:
    print(k,t,x[k,t])

1 1 525.0
1 2 475.0
2 1 475.0
2 2 475.0
3 1 525.0
3 2 575.0
4 1 625.0
4 2 625.0
5 1 525.0
5 2 575.0
6 1 475.0
6 2 475.0
7 1 725.0
7 2 725.0
8 1 575.0
8 2 575.0
9 1 475.0
9 2 475.0
10 1 475.0
10 2 525.0
11 1 575.0
11 2 575.0
12 1 475.0
12 2 475.0
13 1 625.0
13 2 625.0
14 1 725.0
14 2 725.0
15 1 575.0
15 2 525.0
16 1 575.0
16 2 575.0
17 1 575.0
17 2 575.0
18 1 575.0
18 2 625.0
19 1 625.0
19 2 625.0
20 1 475.0
20 2 525.0
21 1 425.0
21 2 425.0
22 1 525.0
22 2 525.0
23 1 575.0
23 2 575.0
24 1 475.0
24 2 425.0
25 1 525.0
25 2 525.0
26 1 475.0
26 2 525.0
27 1 675.0
27 2 675.0
28 1 725.0
28 2 725.0
29 1 725.0
29 2 675.0
30 1 575.0
30 2 625.0
31 1 675.0
31 2 625.0
32 1 475.0
32 2 525.0
33 1 725.0
33 2 675.0
34 1 725.0
34 2 675.0
35 1 525.0
35 2 525.0
36 1 725.0
36 2 675.0
37 1 625.0
37 2 625.0
38 1 725.0
38 2 725.0
39 1 625.0
39 2 625.0
40 1 525.0
40 2 525.0
41 1 625.0
41 2 625.0
42 1 475.0
42 2 475.0
43 1 575.0
43 2 625.0
44 1 625.0
44 2 625.0
45 1 725.0
45 2 725.0
46 1 625.0
46 2 625.0
47 1 4

In [203]:
'''
for k in K:
  for t in T:
    print('scene:',k,'time:',t,Q[k,t].X,'+',bin[k,t].X,'*',v[k,t-1].X,'+',1-bin[k,t].X,'*',u[k,t].X,'-',bin[k,t].X,'*',v[k,t].X,'=',x[k,t])
'''

"\nfor k in K:\n  for t in T:\n    print('scene:',k,'time:',t,Q[k,t].X,'+',bin[k,t].X,'*',v[k,t-1].X,'+',1-bin[k,t].X,'*',u[k,t].X,'-',bin[k,t].X,'*',v[k,t].X,'=',x[k,t])\n"

In [204]:
x[1,1]

525.0

In [205]:
for k in K:
  for t in T:
    print('scene:',k,'time:',t,Q[k,t].X,'+',v[k,t-1].X,'+',u[k,t].X,'-',v[k,t].X,'=',x[k,t])

scene: 1 time: 1 1050.0 + 0.0 + 0.0 - 525.0 = 525.0
scene: 1 time: 2 0.0 + 525.0 + -0.0 - 50.0 = 475.0
scene: 2 time: 1 1050.0 + 0.0 + 0.0 - 575.0 = 475.0
scene: 2 time: 2 0.0 + 575.0 + -0.0 - 100.0 = 475.0
scene: 3 time: 1 1050.0 + 0.0 + 0.0 - 525.0 = 525.0
scene: 3 time: 2 50.0 + 525.0 + -0.0 - -0.0 = 575.0
scene: 4 time: 1 1050.0 + 0.0 + 0.0 - 425.0 = 625.0
scene: 4 time: 2 200.0 + 425.0 + -0.0 - -0.0 = 625.0
scene: 5 time: 1 1050.0 + 0.0 + 0.0 - 525.0 = 525.0
scene: 5 time: 2 50.0 + 525.0 + -0.0 - 0.0 = 575.0
scene: 6 time: 1 1050.0 + 0.0 + 0.0 - 575.0 = 475.0
scene: 6 time: 2 0.0 + 575.0 + -0.0 - 100.0 = 475.0
scene: 7 time: 1 1050.0 + 0.0 + 0.0 - 325.0 = 725.0
scene: 7 time: 2 400.0 + 325.0 + -0.0 - -0.0 = 725.0
scene: 8 time: 1 1050.0 + 0.0 + 0.0 - 475.0 = 575.0
scene: 8 time: 2 100.0 + 475.0 + -0.0 - -0.0 = 575.0
scene: 9 time: 1 1050.0 + 0.0 + 0.0 - 575.0 = 475.0
scene: 9 time: 2 0.0 + 575.0 + -0.0 - 100.0 = 475.0
scene: 10 time: 1 1050.0 + 0.0 + 0.0 - 575.0 = 475.0
scene: 10 

In [206]:
x[k,t]

675.0